# Install and Import Packages, Load Data Set

In [ ]:
!pip install datasets numpy torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.7 MB/s eta 0:00:00


In [ ]:
import json
import pickle
import random
import re

from datasets import load_dataset
import numpy as np
import torch
from transformers import BertTokenizer, BertModel

from google.colab import drive
path = '/content/drive/MyDrive/Text_Mining/BERT-SVM/'
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
raw_data = load_dataset("SemEvalWorkshop/sem_eval_2010_task_8")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.23k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/673k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/231k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2717 [00:00<?, ? examples/s]

# Exploring the Data Set

In [ ]:
raw_data['train'][:5]

{'sentence': ['The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>.',
  'The <e1>child</e1> was carefully wrapped and bound into the <e2>cradle</e2> by means of a cord.',
  'The <e1>author</e1> of a keygen uses a <e2>disassembler</e2> to look at the raw assembly code.',
  'A misty <e1>ridge</e1> uprises from the <e2>surge</e2>.',
  'The <e1>student</e1> <e2>association</e2> is the voice of the undergraduate student population of the State University of New York at Buffalo.'],
 'relation': [3, 18, 11, 18, 12]}

In [ ]:
type(raw_data), raw_data.keys(), len(raw_data['train']), len(raw_data['test']), raw_data['train'][0]['sentence'], raw_data['train'][0]['relation'], raw_data['test'][0]['sentence'], raw_data['test'][0]['relation']

(datasets.dataset_dict.DatasetDict,
 dict_keys(['train', 'test']),
 8000,
 2717,
 'The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>.',
 3,
 'The most common <e1>audits</e1> were about <e2>waste</e2> and recycling.',
 14)

# Set label names

In [ ]:
# extracting the data
data = {
    'train': {
        'tagged': {
            'sentences': [datapoint['sentence'] for datapoint in raw_data['train']],
        },
        'labels': np.array([datapoint['relation'] for datapoint in raw_data['train']]),
    },
    'test': {
        'tagged': {
            'sentences': [datapoint['sentence'] for datapoint in raw_data['test']],
        },
        'labels': np.array([datapoint['relation'] for datapoint in raw_data['test']]),
    },

# creating a dictionary of all the label names
    'label_names': {
        0: 'Cause-Effect(e1,e2)',
        1: 'Cause-Effect(e2,e1)',
        2: 'Component-Whole(e1,e2)',
        3: 'Component-Whole(e2,e1)',
        4: 'Content-Container(e1,e2)',
        5: 'Content-Container(e2,e1)',
        6: 'Entity-Destination(e1,e2)',
        7: 'Entity-Destination(e2,e1)',
        8: 'Entity-Origin(e1,e2)',
        9: 'Entity-Origin(e2,e1)',
        10: 'Instrument-Agency(e1,e2)',
        11: 'Instrument-Agency(e2,e1)',
        12: 'Member-Collection(e1,e2)',
        13: 'Member-Collection(e2,e1)',
        14: 'Message-Topic(e1,e2)',
        15: 'Message-Topic(e2,e1)',
        16: 'Product-Producer(e1,e2)',
        17: 'Product-Producer(e2,e1)',
        18: 'Other',
    },
}

In [ ]:
{datapoint['relation'] for datapoint in raw_data['test']}

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}

In [ ]:
{datapoint['relation'] for datapoint in raw_data['train']}

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}

# Load BERT Tokenizer and Model

In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
bert_model.eval()
device

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

device(type='cuda')

# Tokenize and Encode the Data Set into BERT's Encoding System (WordPiece)

In [ ]:
def encode_data_set(data_set, tokenizer, device):
    # for the train and test data and for each encoding method, tokenise & encode the data, only using the tagged encoding method
    encoding_methods = ['tagged']
    for i in ['train', 'test']:
        for j in encoding_methods:
            tokenize_and_encode(data_set[i][j], tokenizer, device)

def tokenize_and_encode(sub_data_set, tokenizer, device):
    # pass the sentences into the tokenizer, get the encoding ids and attention mask and convert them to the appropriate device type
    tokens = tokenizer(sub_data_set['sentences'], padding=True, truncation=True, return_tensors="pt")
    sub_data_set['encodings'] = tokens['input_ids'].to(device)
    sub_data_set['attention_mask'] = tokens['attention_mask'].to(device)

In [ ]:
encode_data_set(data, tokenizer, device)

In [ ]:
for i in ['train', 'test']:
    for j in ['tagged']:
        for k in ['sentences', 'encodings', 'attention_mask']:
            print(data[i][j][k][100])

Flowers are reborn from the soil and the climate begins to warm, all set to a <e1>score</e1> improvised by an <e2>orchestra</e2> of birds.
tensor([  101,  4870,  2024, 24910,  2013,  1996,  5800,  1998,  1996,  4785,
         4269,  2000,  4010,  1010,  2035,  2275,  2000,  1037,  1026,  1041,
         2487,  1028,  3556,  1026,  1013,  1041,  2487,  1028, 19641,  2011,
         2019,  1026,  1041,  2475,  1028,  4032,  1026,  1013,  1041,  2475,
         1028,  1997,  5055,  1012,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
   

# Get the Sentence Embedding Vectors from BERT

In [ ]:
def embed_data_set(data_set, model, step_size=500):

  # Set the embeddings for  the train and test data, and for each encoding method. Only the tagged encoding method is used.

    encoding_methods = ['tagged']
    for i in ['train', 'test']:
        for j in encoding_methods:
            set_embeddings(data_set[i][j], model, step_size)


def set_embeddings(sub_data_set, model, step_size=250):

    # get the token embeddings and find their mean across each sentence to get the embeddings of the sentences.
    # It uses less memory to get the embeddings for sentences in batches and then concatenate them. A batch step size of 250 works well.
    # This process takes a lot of time and batching also allows printing progress updates.

    mean_embeddings = []
    for i in range(0, len(sub_data_set['encodings']), step_size):
        with torch.no_grad():
            outputs = bert_model(
                sub_data_set['encodings'][i:i+step_size],
                attention_mask=sub_data_set['attention_mask'][i:i+step_size]
            ).last_hidden_state
        mean_embeddings.append(outputs.mean(dim=1))
        print(i, end=' ')
    print()
    sub_data_set['mean_embeddings'] = torch.cat(mean_embeddings, dim=0)


In [ ]:
embed_data_set(data, bert_model, step_size=250)

0 250 500 750 1000 1250 1500 1750 2000 2250 2500 2750 3000 3250 3500 3750 4000 4250 4500 4750 5000 5250 5500 5750 6000 6250 6500 6750 7000 7250 7500 7750 
0 250 500 750 1000 1250 1500 1750 2000 2250 2500 


In [ ]:
for i in ['train', 'test']:
    for j in ['tagged']:
        for k in ['sentences', 'encodings', 'attention_mask']:
            print(data[i][j][k][100])
        for k in ['mean_embeddings']:
            print(data[i][j][k][100].shape)

Flowers are reborn from the soil and the climate begins to warm, all set to a <e1>score</e1> improvised by an <e2>orchestra</e2> of birds.
tensor([  101,  4870,  2024, 24910,  2013,  1996,  5800,  1998,  1996,  4785,
         4269,  2000,  4010,  1010,  2035,  2275,  2000,  1037,  1026,  1041,
         2487,  1028,  3556,  1026,  1013,  1041,  2487,  1028, 19641,  2011,
         2019,  1026,  1041,  2475,  1028,  4032,  1026,  1013,  1041,  2475,
         1028,  1997,  5055,  1012,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
   

# Import the SVM and Metrics from scikit-learn

In [ ]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC, LinearSVC

# Convert Torch Tensors to Numpy Arrays
The scikit-learn SVC works with numpy arrays so we need to convert the data to that format

In [ ]:
def conv_to_np(data_set):
    np_data = {}
    for i in ['train', 'test']:
        np_data[i] = {}
        np_data[i]['labels'] = data[i]['labels']
        for j in ['tagged']:
            np_data[i][j] = {}
            for k in ['mean_embeddings']:
                np_data[i][j][k] = data[i][j][k].cpu().numpy()
    return np_data

In [ ]:
np_data = conv_to_np(data)

# Final Train and Test Data

In [ ]:
encoding_method = 'tagged'
embedding_method = 'mean_embeddings'
X_train = np_data['train'][encoding_method][embedding_method]
Y_train = np_data['train']['labels']
X_test = np_data['test'][encoding_method][embedding_method]
Y_test = np_data['test']['labels']
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((8000, 768), (8000,), (2717, 768), (2717,))

# Grid Search Cross Validation for Hyperparameter Tuning

In [ ]:
param_dict = {
    'C': [0.01, 0.1, 1, 10],
    'class_weight': ['balanced', None]
}

In [ ]:
# hyperparameter tuning
grid_search = GridSearchCV(LinearSVC(), param_dict, cv=5, scoring='accuracy', verbose=3)
grid_search.fit(X_train, Y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[CV 1/5] END .....C=0.01, class_weight=balanced;, score=0.564 total time=   9.5s
[CV 2/5] END .....C=0.01, class_weight=balanced;, score=0.583 total time=   9.6s
[CV 3/5] END .....C=0.01, class_weight=balanced;, score=0.600 total time=   9.5s
[CV 4/5] END .....C=0.01, class_weight=balanced;, score=0.554 total time=   8.9s
[CV 5/5] END .....C=0.01, class_weight=balanced;, score=0.481 total time=   9.4s
[CV 1/5] END .........C=0.01, class_weight=None;, score=0.603 total time=   9.5s
[CV 2/5] END .........C=0.01, class_weight=None;, score=0.591 total time=   9.5s
[CV 3/5] END .........C=0.01, class_weight=None;, score=0.607 total time=   8.9s
[CV 4/5] END .........C=0.01, class_weight=None;, score=0.583 total time=   9.6s
[CV 5/5] END .........C=0.01, class_weight=None;, score=0.499 total time=   9.1s
[CV 1/5] END ......C=0.1, class_weight=balanced;, score=0.620 total time=  11.7s
[CV 2/5] END ......C=0.1, class_weight=balanced;, score=0.623 total time=  11.5s
[CV 3/5] END ......C=0.1, cl

# Evaluate Best Model on Test Data

In [ ]:
svm_best_model = grid_search.best_estimator_
test_accuracy = svm_best_model.score(X_test, Y_test)
print("Test Set Accuracy:", test_accuracy)
y_pred = svm_best_model.predict(X_test)

precision = precision_score(Y_test, y_pred, average='weighted')
print("Test Set Precision:", precision)
recall = recall_score(Y_test, y_pred, average='weighted')
print("Test Set Recall:", recall)
f1 = f1_score(Y_test, y_pred, average='weighted')


test_f1_scores = f1_score(Y_test, y_pred, average=None)
print("Test Set F1:", test_f1_scores)

print(classification_report(Y_test, y_pred))

Test Set Accuracy: 0.6426205373573795
Test Set Precision: 0.6330508167807224
Test Set Recall: 0.6426205373573795
Test Set F1: [0.72103004 0.73706897 0.69090909 0.56976744 0.8089172  0.56410256
 0.78219396 0.         0.67213115 0.70588235 0.3        0.56387665
 0.44827586 0.73443983 0.7505618  0.65909091 0.5915493  0.53941909
 0.304414  ]
              precision    recall  f1-score   support

           0       0.85      0.63      0.72       134
           1       0.63      0.88      0.74       194
           2       0.68      0.70      0.69       162
           3       0.51      0.65      0.57       150
           4       0.79      0.83      0.81       153
           5       0.56      0.56      0.56        39
           6       0.73      0.85      0.78       291
           7       0.00      0.00      0.00         1
           8       0.59      0.78      0.67       211
           9       0.65      0.77      0.71        47
          10       0.33      0.27      0.30        22
          1

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

# Save Best Model

In [ ]:
with open(path + 'Models/svm_best_model.pickle', 'wb') as fp:
    pickle.dump(svm_best_model, fp)

# Experiment with Model
This function is the same as in the interactive demo notebook. It is used here so we can experiment with the model.

In [ ]:
def predict_relation(sentence, tokenizer, bert_model, svm_model, label_names, device):
    # Tokenize the input sentence
    encoded = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")

    # Move tensors to the correct device (GPU or CPU)
    input_ids = encoded["input_ids"].to(device)
    attention_mask = encoded["attention_mask"].to(device)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = bert_model(input_ids, attention_mask=attention_mask)

    sentence_embedding = outputs.last_hidden_state.mean(dim=1)
    X_new = sentence_embedding.cpu().numpy()

    # Predict label using the SVM model
    predicted_label_idx = svm_model.predict(X_new)[0]

    # Convert numeric label back to relation name
    predicted_relation = label_names[predicted_label_idx]

    return predicted_relation


In [ ]:
predict_relation('The <e1>bottle</e1> was filled with <e2>water</e2> and placed on the table.', tokenizer, bert_model, svm_best_model, data['label_names'], device)

'Content-Container(e1,e2)'

In [ ]:
# y_pred[:50], y_train[:50]